In [1]:
import os
import re
import shutil
import matplotlib.pyplot as plt
from PIL import Image
import torchvision
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

In [2]:
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
preprocess = weights.transforms()
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.7)
_ = model.eval()

In [11]:
size = 15
source_path = "./data/"
dest_path = "./result/faster-rcnn-objects/" + str(size) + "/"
if os.path.isdir(dest_path):
    shutil.rmtree(dest_path)
os.mkdir(dest_path)

In [12]:
for folder in os.listdir(source_path):
    if folder == ".DS_Store":
        continue
    for picture in os.listdir(os.path.join(source_path, folder)):
        if picture == ".DS_Store":
            continue
        index = re.match(r'(\d*)_(\d*)x(\d*).jpg', picture).group(1)
        img = Image.open(os.path.join(source_path, folder, picture))
        img = img.resize((size, size), Image.NEAREST)
        trans_totensor = torchvision.transforms.ToTensor()
        img = trans_totensor(img)
        batch = [preprocess(img)]
        prediction = model(batch)[0]
        labels = [weights.meta["categories"][i] for i in prediction["labels"]]
        filename = open(dest_path + index + '.txt', 'w')
        for p, [xmin, ymin, xmax, ymax], label in zip(prediction['scores'].tolist(), prediction['boxes'].tolist(), labels):
            transformed_label = re.split(r' ', label)[-1]
            xmin_transformed = xmin / size * 224
            xmax_transformed = xmax / size * 224
            ymin_transformed = ymin / size * 224
            ymax_transformed = ymax / size * 224
            filename.write(f'{transformed_label} {p:.3f} {xmin_transformed:.3f} {ymin_transformed:.3f} {xmax_transformed:.3f} {ymax_transformed:.3f}\n')
        filename.close()